In [ ]:
import csv
import ast
import requests
import numpy as np
import time
from scipy.spatial.distance import cosine

api_key = "**********************************"

def get_embeddings(text):

    headers = {"Authorization": f"Bearer {api_key}"}
    data = {
        "input": text,
        "model": "text-embedding-3-large"
    }
    response = requests.post("https://api.openai.com/v1/embeddings", headers=headers, json=data)
    if response.status_code == 200:
        embedding = response.json()['data'][0]['embedding']
        return np.array(embedding)
    else:
        print(f"Error with text: {text[:30]}... Status code: {response.status_code}")
        return None

def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

import csv

file_path = '/content/InfluencersSyntheticVsRealTopics.csv'
topics = []

with open(file_path, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        platform, topic_info = row[0], row[1]
        topic_index, keywords_str = topic_info.split(':', 1)
        keywords_list_str = keywords_str[keywords_str.find('['):keywords_str.rfind(']')+1]
        try:
            topic_keywords = ast.literal_eval(keywords_list_str.strip())
        except SyntaxError as e:
            print(f"Error parsing keywords for topic {topic_index}: {e}")
            topic_keywords = []
        topics.append((platform, topic_index.strip(), " ".join(topic_keywords)))


for i, (platform, topic_index, topic_text) in enumerate(topics):
    embedding = get_embeddings(topic_text)
    if embedding is not None:
        topics[i] += (embedding,)
    time.sleep(0.1)

visited_pairs = set()



import numpy as np
from scipy.spatial.distance import cosine


def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

num_topics = len(topics)
similarity_matrix = np.zeros((num_topics, num_topics))

# Calculate similarity between all pairs of topics
for i in range(num_topics):
    for j in range(i, num_topics):
        if i == j:
            similarity_matrix[i][j] = 1.0
        else:
            similarity = cosine_similarity(topics[i][-1], topics[j][-1])
            similarity_matrix[i][j] = similarity_matrix[j][i] = similarity

for i in range(num_topics):
    for j in range(i + 1, num_topics):
        platform_i = topics[i][0]
        platform_j = topics[j][0]

# print(similarity_matrix)



In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd



topic_identifiers = [f"{topic[0]}: {topic[1]}" for topic in topics]
df_similarity = pd.DataFrame(similarity_matrix, index=topic_identifiers, columns=topic_identifiers)

excel_file_path = '/content/InfluencersSyntheticVsRealTopics_similarity_matrix.xlsx'
df_similarity.to_excel(excel_file_path, sheet_name='Similarity')

print(f"Similarity matrix has been saved to {excel_file_path}.")


Similarity matrix has been saved to /content/InfluencersSyntheticVsRealTopics_similarity_matrix.xlsx.
